# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Firstly, import the libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

### scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [ ]:
from bs4 import BeautifulSoup
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('ISO_3166-1_alpha-2.html', 'w') as fo:
    fo.write(article)
    
soup = BeautifulSoup(article, 'html.parser')

### obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe df

In [ ]:
tables = soup.find_all('table', class_='sortable')
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

In [ ]:
df = pd.DataFrame(columns=headings)
i=0

for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighbourhood = [td.text.strip() for td in tds[:3]]
        
        # Ignore cells with a borough that is Not assigned.
        if 'Not assigned' in borough:
            continue
        # If a cell has a borough but a Not assigned neighborhood, 
        # then the neighborhood will be the same as the borough. 
        if 'Not assigned' in neighbourhood:
            neighbourhood = borough
        df.loc[i] = [postcode, borough, neighbourhood]
        i=i+1

In [ ]:
from functools import reduce

def to_set(x):
    setx = set(x)
    strs = ', '.join(setx)   
    return strs

#grouped = df.groupby("Postcode").agg({'Neighbourhood': to_set})

grouped = df.groupby("Postcode").agg({'Borough':'first', 'Neighbourhood': to_set})

df2 = df.drop(columns='Neighbourhood')
grouped = grouped.reset_index()

#grouped = pd.merge(df2, grouped, how='inner', on='Postcode')

    
#grouped = pd.DataFrame({'Qty_cnt' : df['Borough'], grouped]

grouped[['Postcode','Borough','Neighbourhood']]

### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
grouped.shape

### Part 2:  get the latitude and the longitude coordinates of each neighborhood.

In [ ]:
#!conda install -c conda-forge geocoder  #install geocoder for the 1st run
import geocoder # import geocoder

In [ ]:
# Comment:
# I tried to use geocoder, but the API seems not working, 
# so I choose to use the csv file instead
#
#nbh = 'Malvern, Rouge'
#postal_code = 'M1B'
#g = geocoder.google(nbh)
#lat_lng_coords = g.latlng

### Import lat_lng_coords CSV, and show the complete dataframe

In [ ]:
df_ll = pd.read_csv("https://cocl.us/Geospatial_data")

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
g2 = grouped[['Postcode','Borough','Neighbourhood']]
d2 = df_ll.rename(index=str, columns={"Postal Code": "Postcode"})
g2 = pd.merge(d2, g2, how='inner', on='Postcode')

g2 = g2[['Postcode','Borough','Neighbourhood', 'Latitude', 'Longitude']]
g2


### Part 3: Cluster Neighborhoods

#### Define Foursquare Credentials and Version

In [ ]:
# The code was removed by Watson Studio for sharing.

#### Locate the latitude and longitude of Toronto City

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#### Create a map of Toronto with neighborhoods superimposed on top

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = g2

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Explore and cluster the neighborhoods in Toronto. 
#### (We work with only boroughs that contain the word Toronto)

In [ ]:
myList = []
for b in neighborhoods['Borough']:
    myList.append('Toronto' in b)
    #print('Toronto' in b)
    
toronto_data = neighborhoods[myList].reset_index(drop=True)
toronto_data.head()

#### Test with the first Neighbourhood in Toronto Data

In [ ]:
toronto_data.loc[0, 'Neighbourhood']

In [ ]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### Next, get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [ ]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL

results = requests.get(url).json()
results

#### We read all the information is in the items key of Foursquare API:

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### and we restructure the data into dataframe:

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

#### Hence, we know the number of venues obtained from Foursquare:

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

#### In further analysis, we create a function to repeat the same process to all the neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Then, we run the above function on each neighborhood and create a new dataframe called toronto_venues

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

#### The sizes of the toronto_venue data is:

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

#### The number of venues were returned for each neighborhood as below:

In [ ]:
toronto_venues.groupby('Neighbourhood').count()

#### The number of unique categories is shown below:

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

#### Next, we further analysis the neighbourhood:

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()



#### The new dataframe size is:

In [ ]:
toronto_onehot.shape

#### And then, we group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

#### The new dataframe size is confirmed as:

In [ ]:
toronto_grouped.shape

#### We then print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### We then sort the venues in descending order, and put into a pandas dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Finally, we run k-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

#### Finally, we visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Follow up: Cluster examination

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]